In [1]:
import os
import sys
os.getcwd()

'c:\\Users\\ebaca\\Desktop\\Phys 417\\Final Project - HEP Tagging'

In [108]:
# importing libraries & making torch.device object for GPU

# neural network packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.rnn as rnn_utils

from torch import Tensor
from torch.nn import Transformer
from torch.utils.data import Dataset, DataLoader, TensorDataset, Subset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pack_sequence, pad_sequence
from torch.utils.data.sampler import SubsetRandomSampler

# data packages
import numpy as np
import math
import pandas as pd
import sklearn.preprocessing as prep
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import fndict as fd

# visual packages
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tqdm
import warnings
import pprint as pp

# Create a torch.device object to tell pytorch where to store your tensors: cpu or gpu
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [94]:
files = shuffle([f for f in os.listdir('../../PHYS417_Project/data')])

test_index = 0
data = []

for i, f in enumerate(files[:100]):
    with np.load('../../PHYS417_Project/data/' + f) as data_load:
        x_load = data_load['x']
        y_load = data_load['y']
        data.append([x_load, y_load])

        if i == test_index:
            print(x_load.shape, y_load, '\n')

data = np.array(data, dtype=object)

'''
the dataset then contains these dimensions:
data[event index][event information][event constituents]

event index: 
data[i] = [event1, event2, ..., eventn]
- index for a specific event in a set number of sampled events
- number of samples stays the same throughout a training session

event information:
data[i][0] = [[constituent1], [constituent2], ..., [constituentn]]
data[i][1] = [one-hot encoded jet tag for 5 categories]
- 0 for a variable number of constituents and their 5 properties (data['x'] of one event file)
- 1 for the jet tag (represented as a one-hot encoded vector) (data['y'] of one event file)
- always either 0 or 1, doesn't change ever

event constituents: 
data[i][0][constituentn] = [momentum, eta, phi, energy, distance]
- varying number n for each event[i][0][n]
- each represents a row of the nx5 matrix in event[i][0]

'''

class ParticleDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x, y = self.data[idx]
        
        # Convert x to a list of tensors
        x_tensors = [torch.tensor(seq, dtype=torch.float32) for seq in x]
        
        # Pack the sequence of tensors
        packed_x = pack_sequence(x_tensors, enforce_sorted=False)
        
        return packed_x, y

(43, 5) [0 0 0 1 0] 



In [ ]:
print(f"Final shape of data_array: {data.shape}", f"\nShape X (features) {data[:, 0].shape}", f"\nShape Y (targets) {data[:, 1].shape}\n")

print(data[test_index, 0].shape, data[test_index, 1]) # [data element index][0 for data_load['x'], 1 for data_load['y']][constituent index]

# class ParticleDataset(Dataset):
#     def __init__(self, data):
#         self.data = data
        
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, idx):
#         # Return a tuple containing features (x) and targets (y)
#         return self.data[idx][0], self.data[idx][1]

In [191]:
# Define your model architecture
class JetClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(JetClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

# custom PyTorch dataset (for DataLoader)
class JetDataset(Dataset):
    def __init__(self, files):
        self.files = files

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = self.files[idx]
        with np.load(file_path) as data_load:
            x_load = torch.tensor(data_load['x'], dtype=torch.float32)
            y_load = torch.tensor(data_load['y'], dtype=torch.float32)
        return x_load, y_load

# the collate function lets the model handle variable-length sequences
def collate_fn(batch):
    # sorting batch in descending order of # of constituents
    batch.sort(key=lambda x: x[0].size(0), reverse=True)
    inputs, labels = zip(*batch)
    # adding padding to the sequences
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=0)
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=0)
    return inputs_padded, labels_padded

# function to extract numpy arrays from tensors
extractor = lambda x: x.cpu().detach().numpy()


files = shuffle([os.path.join('../../PHYS417_Project/data', f) for f in os.listdir('../../PHYS417_Project/data')])

# splitting into training/testing sets
train_files = files[:int(0.7*len(files))]
test_files = files[int(0.7*len(files)):]

print(f"{len(train_files)} Training Files  + {len(test_files)} Testing Files  =  {len(files)} Total Files\n")

# make datasets into DataLoader objects and apply collate_fn
train_dataset = JetDataset(train_files)
test_dataset = JetDataset(test_files)

train_loader = DataLoader(train_dataset, batch_size=13, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=13, shuffle=False, collate_fn=collate_fn)

for inputs, labels in test_loader:
    print(f'Inputs: {inputs.shape}, Labels: {labels.shape} \n')
    break

# Initialize the model
input_size = 5  # Number of features for each constituent
hidden_size = 128
num_layers = 2
num_classes = 5  # Number of categories for jet classification

model = JetClassifier(input_size, hidden_size, num_layers, num_classes).to(DEVICE)

# Define loss function and optimizer
# criterion = nn.BCEWithLogitsLoss().to(DEVICE)  # Binary Cross Entropy Loss for multi-label classification
criterion = nn.CrossEntropyLoss().to(DEVICE)  # includes softmax layer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

''' 
Training -------------------------------------------------------------------

[batch size][max sequence length][input size/number of features]
'''
num_epochs = 10
for epoch in range(num_epochs):
    for events, tags in train_loader:
        events, onehots = events.to(DEVICE), tags.to(DEVICE)
        tags = torch.argmax(onehots, dim=1) # Convert one-hot encoded labels to single integer labels
        optimizer.zero_grad()
        outputs = model(events)
        loss = criterion(outputs, tags)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    print(f'Inputs: {events.shape}  \nOnehot: {extractor(onehots[:1][0])}  ArgMax Index: {tags[:1].item()}')
    softmax = nn.Softmax(dim=1)
    probabilities = softmax(outputs)
    print(f'Probabilities: {extractor(probabilities[:1][0])}')
    print('\n')


3512 Training Files  + 1506 Testing Files  =  5018 Total Files

Inputs: torch.Size([13, 105, 5]), Labels: torch.Size([13, 5]) 

Epoch [1/10], Loss: 1.0627
Inputs: torch.Size([2, 53, 5])  
Onehot: [0. 0. 0. 0. 1.]  ArgMax Index: 4
Probabilities: [0.4329131  0.02739702 0.01431549 0.03300458 0.4923699 ]


Epoch [2/10], Loss: 1.6374
Inputs: torch.Size([2, 59, 5])  
Onehot: [0. 0. 1. 0. 0.]  ArgMax Index: 2
Probabilities: [0.21412326 0.19273259 0.20080373 0.18838477 0.20395571]


Epoch [3/10], Loss: 1.6131
Inputs: torch.Size([2, 59, 5])  
Onehot: [0. 0. 0. 0. 1.]  ArgMax Index: 4
Probabilities: [0.20148997 0.18933281 0.21059448 0.20099041 0.19759224]


Epoch [4/10], Loss: 1.5641
Inputs: torch.Size([2, 89, 5])  
Onehot: [1. 0. 0. 0. 0.]  ArgMax Index: 0
Probabilities: [0.210441   0.17466551 0.19117518 0.20812348 0.21559484]


Epoch [5/10], Loss: 1.5559
Inputs: torch.Size([2, 86, 5])  
Onehot: [1. 0. 0. 0. 0.]  ArgMax Index: 0
Probabilities: [0.20733067 0.18848772 0.19842994 0.19095495 0.2147

In [209]:
''' 
Evaluation -------------------------------------------------------------------

[batch size][max sequence length][input size/number of features]
'''
def evaluate_model(model, data_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient computation
        for events, onehots in data_loader:
            events, onehots = events.to(device), onehots.to(device)
            tags = torch.argmax(onehots, dim=1)  # Convert one-hot encoded labels to single integer labels

            outputs = model(events)
            loss = criterion(outputs, tags)
            total_loss += loss.item()

            # Get the predicted class by finding the index with the max probability
            softmax = nn.Softmax(dim=1)
            probabilities = softmax(outputs)
            print(probabilities.shape)
            predicted = torch.argmax(probabilities, dim=1)
            total += tags.size(0)
            correct += (predicted == tags).sum().item()

            print(f'Tag Index: {tags[:1].item()}  Predicted Index: {predicted}')
            print(f'Onehot: {extractor(onehots[:1][0])}  Probabilities: {extractor(probabilities[:1][0])}\n')

    average_loss = total_loss / len(data_loader)
    accuracy = correct / total

    return average_loss, accuracy

# Evaluate the model on the test dataset
test_loss, test_accuracy = evaluate_model(model, test_loader, criterion, DEVICE)
print(f'Test Accuracy: {test_accuracy:.4f}')

torch.Size([13, 5])
Tag Index: 0  Predicted Index: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
Onehot: [1. 0. 0. 0. 0.]  Probabilities: [0.20841897 0.19631071 0.18718179 0.20186096 0.20622754]

torch.Size([13, 5])
Tag Index: 0  Predicted Index: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
Onehot: [1. 0. 0. 0. 0.]  Probabilities: [0.20840862 0.19631496 0.18718563 0.20186542 0.20622537]

torch.Size([13, 5])
Tag Index: 0  Predicted Index: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
Onehot: [1. 0. 0. 0. 0.]  Probabilities: [0.20840481 0.19631924 0.18719195 0.20186411 0.20621991]

torch.Size([13, 5])
Tag Index: 0  Predicted Index: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
Onehot: [1. 0. 0. 0. 0.]  Probabilities: [0.20839083 0.19632146 0.18720761 0.20186904 0.20621099]

torch.Size([13, 5])
Tag Index: 4  Predicted Index: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
Onehot: [0. 0. 0. 0. 1.]

In [187]:
''' 
Evaluation -------------------------------------------------------------------

[batch size][sequence length][input size/number of features]
'''
model.eval()
with torch.no_grad():
    correct = []
    total = []
    correct_count = 0
    total_count = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        outputs = model(inputs)
        predicted = torch.round(torch.sigmoid(outputs))  # converting logits to probabilities
        correct.append((predicted == labels).sum().item())
        total_count += labels.size(0) * labels.size(1)  # Total number of labels
        correct_count += (predicted == labels).sum().item()
        print(f'Inputs: {inputs.shape}  Labels: {labels.shape}')
        print(f'Outputs: {outputs.shape} Correct: {correct_count} \n')

    print(f'Accuracy on test set: {(correct_count/total_count) * 100:.2f}%')

Inputs: torch.Size([13, 93, 5])  Labels: torch.Size([13, 5])
Outputs: torch.Size([13, 5]) Correct: 29 

Inputs: torch.Size([13, 81, 5])  Labels: torch.Size([13, 5])
Outputs: torch.Size([13, 5]) Correct: 56 

Inputs: torch.Size([13, 91, 5])  Labels: torch.Size([13, 5])
Outputs: torch.Size([13, 5]) Correct: 85 

Inputs: torch.Size([13, 79, 5])  Labels: torch.Size([13, 5])
Outputs: torch.Size([13, 5]) Correct: 116 

Inputs: torch.Size([13, 94, 5])  Labels: torch.Size([13, 5])
Outputs: torch.Size([13, 5]) Correct: 147 

Inputs: torch.Size([13, 92, 5])  Labels: torch.Size([13, 5])
Outputs: torch.Size([13, 5]) Correct: 178 

Inputs: torch.Size([13, 60, 5])  Labels: torch.Size([13, 5])
Outputs: torch.Size([13, 5]) Correct: 211 

Inputs: torch.Size([13, 92, 5])  Labels: torch.Size([13, 5])
Outputs: torch.Size([13, 5]) Correct: 244 

Inputs: torch.Size([13, 90, 5])  Labels: torch.Size([13, 5])
Outputs: torch.Size([13, 5]) Correct: 269 

Inputs: torch.Size([13, 109, 5])  Labels: torch.Size([13, 

In [105]:
# ---- STEP 1: establishing training features (x) and training targets (y) data -----------------------
# Step 1: Initialize ParticleDataset
particle_dataset = ParticleDataset(shuffle(data))
# particle_dataset[0] means it returns a single element with its 'x' and 'y' values

# Shuffle the indices to split the data randomly
indices = np.arange(len(data))
np.random.shuffle(indices)

# Define the sizes for train, validation, and test sets
train_size = int(0.6 * len(data))
val_size = int(0.2 * len(data))
test_size = len(data) - train_size - val_size

# Split the indices
train_indices = indices[:train_size]
val_indices = indices[train_size:train_size+val_size]
test_indices = indices[train_size+val_size:]

# Define datasets and data loaders for each set
train_dataset = Subset(particle_dataset, train_indices)
val_dataset = Subset(particle_dataset, val_indices)
test_dataset = Subset(particle_dataset, test_indices)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1)
test_loader = DataLoader(test_dataset, batch_size=1)


# # ---- STEP 3: sending to GPU  --------------------------------------------------------------
# print("\n --Sending to GPU--")

# with warnings.catch_warnings(): # booo warnings
#     warnings.simplefilter("ignore")

#     trfeat = torch.tensor(torch.from_numpy(trfeat), dtype=torch.float32).to(DEVICE)
#     trtarget = torch.tensor(torch.from_numpy(trtarget), dtype=torch.float32).to(DEVICE)

#     vafeat = torch.tensor(torch.from_numpy(vafeat), dtype=torch.float32).to(DEVICE)
#     vatarget = torch.tensor(torch.from_numpy(vatarget), dtype=torch.float32).to(DEVICE)
    
#     tefeat = torch.tensor(torch.from_numpy(tefeat), dtype=torch.float32).to(DEVICE)
#     tetarget = torch.tensor(torch.from_numpy(tetarget), dtype=torch.float32).to(DEVICE)

# # Determine lengths of sequences for packed sequences
# trlengths = torch.tensor([len(seq) for seq in trfeat]).to(DEVICE)
# valengths = torch.tensor([len(seq) for seq in vafeat]).to(DEVICE)
# telengths = torch.tensor([len(seq) for seq in tefeat]).to(DEVICE)

(PackedSequence(data=tensor([ 3.4704e-01,  1.1869e-01,  1.1574e-01,  8.2077e-02,  5.5813e-02,
          3.1093e-02,  2.2187e-02,  1.9635e-02,  1.7024e-02,  1.6102e-02,
          1.5947e-02,  1.3207e-02,  1.2627e-02,  1.1193e-02,  1.0204e-02,
          1.0054e-02,  9.0737e-03,  8.0491e-03,  7.7192e-03,  7.5888e-03,
          6.7408e-03,  6.4573e-03,  5.2918e-03,  5.1927e-03,  4.9273e-03,
          4.8966e-03,  4.1454e-03,  3.7508e-03,  3.5925e-03,  3.3263e-03,
          3.1473e-03,  2.8536e-03,  2.6036e-03,  2.4031e-03,  2.3430e-03,
          2.0198e-03,  1.6181e-03,  1.5652e-03,  1.3808e-03,  1.0791e-03,
          9.4074e-04,  7.1847e-04,  4.7826e-04,  4.7442e-04,  1.1651e-04,
          0.0000e+00, -1.0468e-02, -1.6900e-02, -1.0503e-02,  8.4449e-03,
         -3.5364e-03,  6.7515e-14, -1.9937e-03,  8.9245e-03, -2.0778e-02,
         -3.6403e-03,  6.7177e-03, -1.4146e-04, -3.9678e-02,  4.7722e-03,
         -5.3258e-02,  2.9307e-02,  1.4913e-03,  1.7846e-02,  3.3074e-02,
          2.1003e-

In [96]:
# Define the model
class ParticleClassifierLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes):
        super(ParticleClassifierLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x, lengths):
        packed_input = rnn_utils.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
        packed_output, (h_n, c_n) = self.lstm(packed_input)
        output, _ = rnn_utils.pad_packed_sequence(packed_output, batch_first=True)
        idx = (lengths - 1).view(-1, 1).expand(len(lengths), output.size(2)).unsqueeze(1).to(DEVICE)
        last_output = output.gather(1, idx).squeeze(1)
        out = self.fc(last_output)
        return out

# Initialize the model, loss function, and optimizer
model = ParticleClassifierLSTM(
    input_dim = 5, 
    hidden_dim = 100, 
    num_layers = 2, 
    num_classes = 5
).to(DEVICE)

In [100]:
# Assuming you have defined the model, loss function, optimizer, and other necessary components
num_epochs = 50
batch_size = 1
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        # Unpack the packed sequence
        inputs, lengths = nn.utils.rnn.pad_packed_sequence(inputs, batch_first=True)
        
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs, lengths)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # Validate the model
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for inputs, targets in val_loader:
            # Unpack the packed sequence
            inputs, lengths = nn.utils.rnn.pad_packed_sequence(inputs, batch_first=True)
            
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
            outputs = model(inputs, lengths)
            val_loss += criterion(outputs, targets).item()
        
        val_loss /= len(val_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss}')

RuntimeError: 'lengths' argument should be a 1D CPU int64 tensor, but got 2D cpu Long tensor

In [ ]:
epochs = 50
batch_size = 64
learning_rate = 0.001


# Training loop

def train(model, loss_fn, optimizer, epochs, trfeat, trtarget, vafeat, vatarget, trlengths, valengths, batch_size):
    model.train()
    for epoch in range(epochs):
        for i in range(0, len(trfeat), batch_size):
            x_batch = trfeat[i:i+batch_size]
            y_batch = trtarget[i:i+batch_size]
            lengths_batch = trlengths[i:i+batch_size]

            optimizer.zero_grad()
            output = model(x_batch, lengths_batch)
            loss = loss_fn(output, y_batch)
            loss.backward()
            optimizer.step()

        val_loss, val_acc = evaluate(model, loss_fn, vafeat, vatarget, valengths)
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Validation Loss: {val_loss}, Validation Accuracy: {val_acc}')

def evaluate(model, loss_fn, vafeat, vatarget, valengths):
    model.eval()
    with torch.no_grad():
        output = model(vafeat, valengths)
        loss = loss_fn(output, vatarget)
        preds = torch.argmax(output, dim=1)
        accuracy = (preds == vatarget).float().mean()
    return loss.item(), accuracy.item()

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train(model, loss_fn, optimizer, epochs, trfeat, trtarget, vafeat, vatarget, trlengths, valengths, batch_size)
evaluate(model, loss_fn, vafeat, vatarget, valengths)

In [ ]:
test_outputs = classifier(tefeat)
_ignore_, predicted = torch.max(test_outputs, 1)
correct = (predicted == tetarget).float()
accuracy = correct.mean().item()
# print(f'Test Accuracy: {accuracy*100:.3f}%')

# Plot the loss
plt.figure(figsize = (12, 7))

plt.subplot(2, 1, 1)
plt.plot(trlosses, linewidth = 3)
# plt.plot(runner.losses, linewidth = 3)
plt.ylabel("Losses in Training")
plt.annotate(f'Learning Rate: {learning_rate} \nBatch Size: {batch_size} \nLowest Loss: {min(trlosses):.3f}', 
             xy=(0.95, 0.85), xycoords='axes fraction', va='top', ha='right')
sns.despine()

plt.subplot(2, 1, 2)
plt.plot(valosses, linewidth = 3, color = 'gold')
# plt.plot(runner.accuracies, linewidth = 3, color = 'gold')
plt.ylabel("Training Accuracy (Validation)")
plt.annotate(f'Accuracy: {accuracy*100:.3f}%', xy=(0.95, 0.20), xycoords='axes fraction', va='top', ha='right')
sns.despine()